# RSA signatures

Sigurjón Ágústsson

In [72]:
import random
import time
import numpy as np

In [73]:
def load_small_primes():
    pfile =  open("1000primes.txt", "r")
    small_primes = [int(num) for num in pfile.read().split(',')]
    return small_primes

small_odd_primes = load_small_primes()
print("Small odd primes loaded : {}...{}".format(small_odd_primes[:5], small_odd_primes[-3:]))

Small odd primes loaded : [3, 5, 7, 11, 13]...[7901, 7907, 7919]


We start by introducing some helper functions. Most important of which is the modular exponentiation function which uses Montgomery modular multiplication to very quickly compute modular exponentiation.

In [74]:
def even(n):
    """Checks the parity of n"""
    return (n&1) != 1

def pow_n_mod_n(a,n):
    """Naive and very inefficient way of doing modular exponentiation"""
    return ((a**(n-1)) % n)

def modular_exponentiation(a,n):
    """Python's fast way of doing modular exponentiation"""
    return pow(a,(n-1),n)

def divisible_by_small_prime(n, resolution=1000):
    """A function that checks if a number n has one of the 
        1000 first primes as a factor (excluding 2 because 
        it is checked for already)"""
    global small_odd_primes # The array of primes
    for small_prime in small_odd_primes[:resolution]:
        if n == small_prime:
            # In this case n is a small prime number, and we return
            # False to indicate it is not divisible by any of
            # the smaller primes
            return False
        if (n % small_prime) == 0:
            # print("diviseble by small prime")
            return True
    return False

def seconds_to_prefix(sec):
    """Just a function that inputs number of seconds (float) and 
        outputs a string representation with a unit (s, ms or min) """
    if sec < 1:
        ms = sec * 1000
        return "{} ms".format(round(ms,4))
    elif sec < 60:
        return "{} s".format(round(sec,4))
    else:
        min = int(sec) // 60
        rem = seconds_to_prefix(sec-60*min)
        return "{} min and {}".format(min,rem)

Let's demonstrate with a timing example.

In [75]:
bits = 20
# Let n be a random 20 bit integer
n = random.randint((1<<(bits-1)),((1<<bits)-1))
a = random.randint(0xFFFF,n-1) # a very large number but still less than n


ex1_start = time.perf_counter()
print(pow_n_mod_n(a,n))
ex1_end = time.perf_counter()
ex2_start = time.perf_counter()
print(modular_exponentiation(a,n))
ex2_end = time.perf_counter()
t1,t2 = (ex1_end-ex1_start), (ex2_end-ex2_start)
strt1,strt2 = seconds_to_prefix(t1), seconds_to_prefix(t2)
diff = round(t1/t2,2)
print("Using the slow algorithm, it took: {} \
        \nUsing the fast algorithm, it took: {}".format(strt1,strt2))
print("The Montgomery algorithm makes modular exponentiation {} times faster in this case!".format(diff))

590645
590645
Using the slow algorithm, it took: 3.5609 s         
Using the fast algorithm, it took: 0.116 ms
The Montgomery algorithm makes modular exponentiation 30697.57 times faster in this case!


## Prime number generation

Now we are ready to do generate prime numbers. 

In [76]:
def probable_prime(size):
    assert size > 0
    s = size-1
    smallest = 1<<s
    largest = (1<<(s+1))-1
    while True:
        # First pick a random number n of size 'size' bits
        n = random.randint(smallest,largest)
        if even(n) or divisible_by_small_prime(n):
            continue
        a = random.randint(2,n-1)

        mod_computation = modular_exponentiation(a,n)

        if mod_computation == 1:
            # By Fermat's little thm, if the result is not 1
            # then n is not prime, otherwise, it is likely to 
            # be prime
            return n

Let's use this to generate some "small" probably-prime numbers, or pseudoprimes.

In [77]:
for i in range(4):
    bits = 8*(i+1)
    print("{} bits: \t".format(bits), probable_prime(bits))

8 bits: 	 173
16 bits: 	 53003
24 bits: 	 11210629
32 bits: 	 2695857751


In RSA encryption we want much larger primes. Typically 1024 or 3072 bits. That is no obstacle for us, let's generate such pseudoprimes:

In [78]:
t1s = time.perf_counter()
p1024 = probable_prime(1024)
t1e = time.perf_counter()
print(p1024)
print("execution time: {} seconds".format(round((t1e-t1s),3)))

162088618182906247802844439184656953055901271354671673722079595956583648288915734798509559525783212607680839297010899969549181411136844378413322138087245654970321755841486605263890775679164284296660616148460201217733735194005418056694470723421042298592605685629958478083455790909964057477238507741079739467359
execution time: 0.417 seconds


In [79]:
t1s = time.perf_counter()
p3072 = probable_prime(3072)
t1e = time.perf_counter()
print(p3072)
print("execution time: {} seconds".format(round((t1e-t1s),3)))

4849732060247485938946688631522390566123763746121529703819726411811034616153146207954345931479384192368086835697121989073380999713548613791230674124599896545051517999534130988159931571178613905450980589834327351503742523858055166218010197915872523758302870243472270724772952754281053645481467310909891874285044116297146111328970727915915332475849433517262048491344878210703262739552227003960038310457380104148702139167754669372211327274187344299322506715969272759281696782071699179176388306030178034024014511634902725624175233212046083633439644871219402414813904195882501311605485109730508539869800281338353850225646011385949424149795105319953162193447330817684421763161270705583859542449023684338919317065002251407204890774735994874226128862884886842700788213050797903486767364577517599964016022847386552744772925604469505366995586771595889853553276799383986301998837645902735806912006147629855358262633032041978396543488529
execution time: 7.288 seconds


## RSA signatures

In [80]:
e = 3
print(pow(e,-1,500))

167


In [81]:
def find_ed(N,T):
    e = 3
    d = pow(e,-1,T)
    return (e,d)

In [82]:
def generate_keys(size=1024):
    """Function that returns a tuple of public and private keys of given size (in bits)"""
    p,q = probable_prime(size),probable_prime(size)
    N = p*q
    # e is less than T. And it is coprime to T and N
    # e*d mod T has to be 1
    # e is typically 3
    T = N - p - q + 1 # (p-1)(q-1)
    e = 3
    d = pow(e,-1,T)
    return ((N,e),(N,d))

def dummy_keys():
    return ((91, 5), (91, 29))

def encrypt(val, public_key):
    """public key is a tuple containing N,e """
    N,e = public_key
    return pow(val,e,N)

def decrypt(cipher, private_key):
    """Private key is a tuple containing N,d """
    N,d = private_key
    return pow(cipher,d,N)

In [84]:
public_key, secret_key = generate_keys()
# public_key, secret_key = dummy_keys()

In [85]:
public_key

(18473539090594177804303427473636989622335038717179524563741508944976160463216543196803623416890212848363216912313653944930849394976588643307353626494793388701913561901007569837654184936370426662308106064321464387374208325785966215853066414784652118385436628084898380550327671046112985234778267064138917955236296584141117696584399617009602662667478353014587036992132179513896912204877809689086020487427862623723407720417437827266136071860295994914189107185206924206056977848466429496643482724115373821410069544738941963394693524660868474568965176151217795057713276178573634135401136898536719936432754272392587470700477,
 3)

In [86]:
secret_key

(18473539090594177804303427473636989622335038717179524563741508944976160463216543196803623416890212848363216912313653944930849394976588643307353626494793388701913561901007569837654184936370426662308106064321464387374208325785966215853066414784652118385436628084898380550327671046112985234778267064138917955236296584141117696584399617009602662667478353014587036992132179513896912204877809689086020487427862623723407720417437827266136071860295994914189107185206924206056977848466429496643482724115373821410069544738941963394693524660868474568965176151217795057713276178573634135401136898536719936432754272392587470700477,
 1231569272706278520286895164909132641489002581145301637582767262998410697547769546453574894459347523224214460820910262995389959665105909553823575099652892580127570793400504655843612329091361777487207070954764292491613888385731081056871094318976807892362441872326558703355178069740865682318551137609261197015734983206902184192874332876708286300756892953835600366806615165453639182

In [87]:
text = "HELLO WORLD!"

In [88]:
def str2ascii_list(string):
    return [ord(c) for c in string]

def ascii_list2str(alist):
    s = ""
    for c in alist:
        s+=chr(c)
    return s

def encrypt_string(string, public_key):
    ascii_list = [ord(i) for i in string]
    enc_list = [encrypt(j,public_key) for j in ascii_list]
    print(enc_list)
    enc_string  = ""
    for enc in enc_list:
        enc_string += chr(enc)
    return enc_list

def decrypt_string(enc_list,private_key):
    ascii_list = [decrypt(j, private_key) for j in enc_list]
    return ascii_list

In [89]:
text_asc = str2ascii_list(text)
print("original:")
print(text_asc)
print("encrypted:")
enc = [encrypt(c, public_key) for c in text_asc]
print(enc)
print("decrypted_list")
dec_list = [decrypt(c, secret_key) for c in enc]
print(dec_list)
print("result")
res = ascii_list2str(dec_list)
print(res)

original:
[72, 69, 76, 76, 79, 32, 87, 79, 82, 76, 68, 33]
encrypted:
[373248, 328509, 438976, 438976, 493039, 32768, 658503, 493039, 551368, 438976, 314432, 35937]
decrypted_list
[72, 69, 76, 76, 79, 32, 87, 79, 82, 76, 68, 33]
result
HELLO WORLD!


In [90]:
# ASCII has values from 32 to 126
# print("Public,private keys are: ", public_key, secret_key)
for i in range(32,127):
    enc = encrypt(i, public_key)
    dec = decrypt(enc, secret_key)
    if dec == i:
        print(f"{i}, ASCII: '{chr(i)}', Encryption: '{enc}', Decrypting encryption: '{dec}'")
    else:
        print(f"{i}, ASCII: '{chr(i)}', Encryption: '{enc}', Decrypting encryption: '{dec}' - FAIL")

32, ASCII: ' ', Encryption: '32768', Decrypting encryption: '32'
33, ASCII: '!', Encryption: '35937', Decrypting encryption: '33'
34, ASCII: '"', Encryption: '39304', Decrypting encryption: '34'
35, ASCII: '#', Encryption: '42875', Decrypting encryption: '35'
36, ASCII: '$', Encryption: '46656', Decrypting encryption: '36'
37, ASCII: '%', Encryption: '50653', Decrypting encryption: '37'
38, ASCII: '&', Encryption: '54872', Decrypting encryption: '38'
39, ASCII: ''', Encryption: '59319', Decrypting encryption: '39'
40, ASCII: '(', Encryption: '64000', Decrypting encryption: '40'
41, ASCII: ')', Encryption: '68921', Decrypting encryption: '41'
42, ASCII: '*', Encryption: '74088', Decrypting encryption: '42'
43, ASCII: '+', Encryption: '79507', Decrypting encryption: '43'
44, ASCII: ',', Encryption: '85184', Decrypting encryption: '44'
45, ASCII: '-', Encryption: '91125', Decrypting encryption: '45'
46, ASCII: '.', Encryption: '97336', Decrypting encryption: '46'
47, ASCII: '/', Encryptio

In [91]:
encrypted = encrypt_string(text,public_key)
print(encrypted)
decrypted = decrypt_string(encrypted,secret_key)
print(decrypted)

[373248, 328509, 438976, 438976, 493039, 32768, 658503, 493039, 551368, 438976, 314432, 35937]
[373248, 328509, 438976, 438976, 493039, 32768, 658503, 493039, 551368, 438976, 314432, 35937]
[72, 69, 76, 76, 79, 32, 87, 79, 82, 76, 68, 33]


Different way to encode strings

In [92]:
def string_to_num(string):
    num = 0
    for c in string:
        new_char = ord(c)
        num = (num<<7)+new_char
    return num

def num_to_string(num):
    string = ""
    while num>0:
        new_char = 0x7F
        new_char &= num # Now new_char is the ascii value of the last character of the string
        string = chr(new_char) + string # We prepend the new character to the string, since we are going in the reverse
        num = num >> 7
    return string

s = "AB"

string_to_num(s)


8386

In [109]:
public_key, private_key = generate_keys(100) # generate keys of size 100
num_chars = 5
bits = num_chars * 8
largest, lowest = (1<<bits)-1, (1<<(bits-1))
for _ in range(5):
    num = random.randint(lowest,largest)
    s = num_to_string(num)
    e = encrypt(num, public_key)
    d = decrypt(e, private_key)
    ds = num_to_string(d)
    p  = f"{num} = {s} \t ---> {e} \t ---> {d} = {ds}"
    print(p)


1045397365253 = 64'D 	 ---> 1142468418447782963068794552600049277 	 ---> 1045397365253 = 64'D
640953800229 = S^K,% 	 ---> 263317777280064067528397054689408989 	 ---> 640953800229 = S^K,%
836722336110 = -&Bn 	 ---> 585792878428491770336270052478131000 	 ---> 836722336110 = -&Bn
1086472164976 = O6&,p 	 ---> 1282495390895120776583944868701106176 	 ---> 1086472164976 = O6&,p
941251558814 = 27[{ 	 ---> 833906051311083277606039287401057144 	 ---> 941251558814 = 27[{


In [101]:
ord('A')

65

In [45]:
bin(8386)

'0b10000011000010'

In [47]:
chr(0b1000010)

'B'

In [48]:
chr(0b1000001)

'A'

In [49]:
bin(64)

'0b1000000'